# Python Packaging and Unit Testing

- We already know the benefits of packaging our code:
    - testable
    - reusable
    - makes future work _easier_
- If we are going to make a package, we will be writing _unit tests_ for every function we include

- The basic directory structure of a python package is something like this:

```
project
|
|
|__ myPackage
|     |
|     |__ somePython.py
|     |__ __init__.py
|
|__ tests
      |__ test.py
```

- functions and classes will go in the `somePython.py` file
- The `__init__.py` file is a special file designating that the files in the directory are part of a package. 
    - prevents directories with a common name from unintentionally hiding valid modules that occur later (deeper) on the module search path.
    - In the simplest case, `__init__.py` can just be an empty file, but it can also execute initialization code for the package
- tests for functions will be contained in the `tests` directory, either entirely in one file, or spread across several files which have `test` contained in the name

## Unit Testing
- For every function we write, we feed it inputs and want to make sure it works the way we expect.
- When we modify/improve our code, we want to trust the changes we made didn't break any expected behavior.
- When we design functions, we also need to write unit tests to check the inputs and outputs of them, which can be tested every time we make changes to our code

### Components of a unit test
- "unit" is defined as an isolated test case that consists of the following components:
    - "fixture" (e.g. function, class method, or data file)
    - an action on the fixture (e.g. calling a function with a particular input)
    - an expected outcome (the expected return value of the function)
    - the actual outcome (the actual return value of the function)
    - a verification message (a report whether the return matches the expected or not)

There are three popular packages for testing in python:

#### `unittest`
- part of the Python Standard Library
- "base" testing framework in python

#### `nosetest`
- alternative testing framework
- crawls a subdirectory tree while looking for `.py` files that start with the naming prefix "test".
    - those files will be executed by the unit testing framework
- slightly faster than `py.test`, only searches subdirectories that start with "test"

#### `py.test`
- stands out due to its ease of use, simpler syntax
     - `assert` instead of numerous `assertSomething` commands in `unittest`
- similar to nosetest, executes `.py` files named in the form `test_*.py` or `*_test.py` in the current directory and its subdirectories
- to install: 
```
pip install pytest
```
- In these exercises, I'll be using the `pytest` package to test functions

### Basic pytest usage
- Let's create a file called `test_capitalize.py` and inside it write a function called `capital_case`, which should take a string as its argument, and return a capitalized version of the string.
- We will also write a test, `test_capital_case` to make sure the function returns what we expect.
    - we prefix our test function names with `test_`, since this is what pytest expects our test functions to be named.

In [8]:
# test_capitalize.py

def capital_case(x):
    return x.capitalize()

def test_capital_case():
    assert capital_case('semaphore') == 'Semaphore'

- pytest uses a plain `assert` statement, which is clear and easier to remember compared to the number of `assertSomething` functions found in `unittest`.

- we can then run the `pytest` command in the directory containing this file to execute the test.
- this test should pass, but we have not tested _everything_
    - what if a number is passed in as the argument?
        - We can add the `pytest.raises` helper, which asserts that our function should raise a `TypeError` in case the argument passed is not a string

In [12]:
# test_capitalize.py

import pytest

def test_capital_case():
    assert capital_case('semaphore') == 'Semaphore'

def test_raises_exception_on_non_string_arguments():
    with pytest.raises(TypeError):
        capital_case(9)

- Running the tests would now fail:
```
def capital_case(x):
>       return x.capitalize()
E       AttributeError: 'int' object has no attribute 'capitalize'
```

## Example: Creating a python package using more advanced pytest features

- Let's run through an example of creating a package and using `py.test` to test its functions:
   - This package will do some basic arithmetic using `wallet` that simulates users adding or spending money in their wallet.  It will be modeled as a class with two methods: `spend_cash` and `add_cash`.

- The first thing we want to do is set up the the directory structure.  It should look like this:

```
pywallet-example
|
|
|__ pywallet
|     |
|     |__ wallet.py
|     |__ __init__.py
|
|__ tests
|      |__ test_wallet.py
|
|__ setup.py

```

- Note we have added a `setup.py` file in the top level of the repo.  This contains details on the package, more detail on this later.

- Now add some functions to a `wallet.py` file.  Here we group the `spend_cash` and `add_cash` functions in the `Wallet` class.

In [13]:
# wallet.py

class InsufficientAmount(Exception):
    pass


class Wallet(object):

    def __init__(self, initial_amount=0):
        self.balance = initial_amount

    def spend_cash(self, amount):
        if self.balance < amount:
            raise InsufficientAmount('Not enough available to spend {}'.format(amount))
        self.balance -= amount

    def add_cash(self, amount):
        self.balance += amount

- We can then add a custom exception, `InsufficientAmount`, which will be raised when we try to spend more money than we have in the wallet.
- The initial amount in the wallet defaults to 0, which is also set as the balance
- The `spend_cash` method checks if there is a sufficient balance, and if there isn't it raises the exception
- the `add_cash` method simply adds the amount to the current wallet balance

Now it is time to come up with some tests to make sure our functions are performing the way we want to.  We put these tests in `test_wallet.py`:

In [16]:
# test_wallet.py

import pytest
from wallet import Wallet, InsufficientAmount


def test_default_initial_amount():
    wallet = Wallet()
    assert wallet.balance == 0

def test_setting_initial_amount():
    wallet = Wallet(100)
    assert wallet.balance == 100

def test_wallet_add_cash():
    wallet = Wallet(10)
    wallet.add_cash(90)
    assert wallet.balance == 100

def test_wallet_spend_cash():
    wallet = Wallet(20)
    wallet.spend_cash(10)
    assert wallet.balance == 10

def test_wallet_spend_cash_raises_exception_on_insufficient_amount():
    wallet = Wallet()
    with pytest.raises(InsufficientAmount):
        wallet.spend_cash(100)

### Pytest Fixtures
- At this point, all of these tests should be passing when we run `pytest`
- There is a lot of repetition in these tests though, so let's refactor them using fixtures
    - these are created using the `@pytest.fixture` decorator
    - test functions that require fixtures should accept them as arguments

- We can create a fixture for an `empty_wallet` and a `wallet` with a balance of 20.  Add docstrings to describe what each one is:

In [ ]:
# test_wallet.py

import pytest
from wallet import Wallet, InsufficientAmount

@pytest.fixture
def empty_wallet():
    '''Returns a Wallet instance with a zero balance'''
    return Wallet()

@pytest.fixture
def wallet():
    '''Returns a Wallet instance with a balance of 20'''
    return Wallet(20)

def test_default_initial_amount(empty_wallet):
    assert empty_wallet.balance == 0

def test_setting_initial_amount(wallet):
    assert wallet.balance == 20

def test_wallet_add_cash(wallet):
    wallet.add_cash(80)
    assert wallet.balance == 100

def test_wallet_spend_cash(wallet):
    wallet.spend_cash(10)
    assert wallet.balance == 10

def test_wallet_spend_cash_raises_exception_on_insufficient_amount(empty_wallet):
    with pytest.raises(InsufficientAmount):
        empty_wallet.spend_cash(100)

- Utilizing fixtures helps us follow the DRY principles

### Parametrized Test Functions
- The above tests were all individual.  The next step is to test various combinations
    - e.g. If the initial balance is 30, and I spend 20, then add 100, how much should my balance be?

- Pytests provides parametrized test functions, using the `@pytest.mark.parametrize` decorator:

In [20]:
# test_wallet.py

@pytest.fixture
def my_wallet():
    '''Returns a Wallet instance with a zero balance'''
    return Wallet()

@pytest.mark.parametrize("earned,spent,expected", [
    (30, 10, 20),
    (20, 2, 18),
])
def test_transactions(my_wallet, earned, spent, expected):
    my_wallet.add_cash(earned)
    my_wallet.spend_cash(spent)
    assert my_wallet.balance == expected

- The test function marked with the decorator is run once for each set of parameters
    - The test will run the first time with the `earned` = 30, `spent` = 10, and `expected` = 20

## Bundling everything together into a package
- The goal is to be able to share the package and its functions with others.  We can do this by hosting it on GitHub/GitLab
    - This example will be hosted on GitLab
        - We'll want to add a `README.md` to describe the package and how to install it
        - We'll also want to set this up with CI to automatically run our tests when new code is pushed

- This will be the directory structure:

```
pywallet-example
|
|
|__ pywallet
|     |
|     |__ wallet.py
|     |__ __init__.py
|
|__ tests
|      |__ test_wallet.py
|
|__ setup.py
|
|__ README.md
|
|__ .gitlab-ci.yml

```

- The `__init__.py` is just an empty file to indicate that the functions in `pywallet` are in a package.

### Describing the package
- The package is described in the `setup.py` file, for this example, mine looks like this:

```
from setuptools import setup, find_packages

setup(
    name='pywallet-example',
    version='0.1',
    packages=find_packages(exclude=['tests*']),
    license='none',
    description='An example python package using wallet that simulates users adding or spending money',
    long_description=open('README.md').read(),
    install_requires=[],
    url='https://gitlab.com/scheidec/pywallet-example',
    author='Caleb Scheidel',
    author_email='caleb@methodsconsultants.com'
)
```

### Setting up CI
- The `.gitlab-ci.yml` file will orchestrate the tests to run each time new code is pushed to GitLab.

- For this example, mine looks like this:

```    
image: python:latest

test:
  stage: test
  script: 
  - pip install pytest
  - python -m pytest -v
```

- See the full example here: https://gitlab.com/scheidec/pywallet-example

### Adding Extra/Test Data
- Some packages may need to contain some data to use for tests
- If you need to include some data files, you will also need to write a `MANIFEST.in` file at the top level of the project, and put the data files in a `data` directory under the package directory.


- The structure should look something like this:

```
pywallet-example
|
|
|__ pywallet
|     |
|     |__ data
|     |     |
|     |     |__ test_data.csv
|     |
|     |__ wallet.py
|     |__ __init__.py
|
|__ tests
|     |__ test_wallet.py
|
|__ setup.py
|
|__ MANIFEST.in
|
|__ README.md
```

- For each data file, include a line in `MANIFEST.in` that gives the path to the file you want to include:
    
```
include pywallet/data/test_data.csv
```
- All of the files listed like the above in `MANIFEST.in` will be included with the package when others install it.